In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing all the imporatant libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
train.head()

In [ ]:
test = pd.read_csv("/kaggle/input/titanic/test.csv")
test.head()

In [ ]:
train.info()

Looking out for null values

In [ ]:
train.isnull()
#if not null-> false
#if null-> true

Analyzing the **missing values**

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(train.isnull())

In the above heatmap we can makeout that the white color signifies all the missing values!


Count of who survived, and who didnt. for classification problem, its better to see the ratio of the actual target label



In [ ]:
sns.countplot(x='Survived',data=train)

In [ ]:
sns.countplot(x='Survived',data=train,hue='Sex')

In [ ]:
sns.countplot(x='Survived',data=train,hue='Pclass')

In [ ]:
sns.displot(train['Age'].dropna(),bins=30)
#dropna()-> to remove missing values
# ALT train['Age'].plot.hist()


In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(train.isnull())

Filling in the missing age data inplace of just dropping it off

In [ ]:
sns.boxplot(x='Pclass',y='Age',data=train)

avg. age values can be used to fill up the emtpy boxes



In [ ]:
def new_age(cols):
    Age=cols[0]
    Pclass=cols[1]
    
    if pd.isnull(Age):
        if Pclass==1:
            return 37
        elif Pclass==2:
            return 29
        else:
            return 24
    else:
        return Age
#avg value is written by analyzing the box plot average

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(train.isnull())

In [ ]:
train['Age']=train[['Age','Pclass']].apply(new_age,axis=1)

In [ ]:
sns.heatmap(train.isnull())

We can directly drop off the cabin column, since its just useless

In [ ]:
train.drop("Cabin",axis=1,inplace=True)

In [ ]:
sns.heatmap(train.isnull())

**DATA IS CLEANED NOW!!**

In [ ]:
train.info()

In [ ]:
train.head()

## Converting Categorical Features 

We'll need to convert categorical features to dummy variables using pandas! Otherwise our machine learning algorithm won't be able to directly take in those features as inputs.

In [ ]:
sex=pd.get_dummies(train['Sex'],drop_first=True)
embark = pd.get_dummies(train['Embarked'],drop_first=True)

In [ ]:
train.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)

In [ ]:
train = pd.concat([train,sex,embark],axis=1)
train.head()

**OUr data is ready for our model**

In [ ]:
sex.head()

In [ ]:
embark.head()

# Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train.info()

In [ ]:
train.columns

In [ ]:
X=train[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'male', 'Q',
       'S']]
y=train['Survived']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.50, 
                                                    random_state=101)

# Training and Prediction

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LR=LogisticRegression(max_iter=1000)


In [ ]:
LR.fit(X_train, y_train)

In [ ]:
predictions = LR.predict(X_test).astype(int)
Prediction = pd.DataFrame({'Survived': predictions})
Prediction.shape

# Evaluation

We can check precision,recall,f1-score using classification report!

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(LR, X_test, y_test)

In [ ]:
from sklearn.metrics import accuracy_score
print(f' Accuracy of our model is=> {accuracy_score(y_test, predictions)}')

In [ ]:
#Prediction = pd.DataFrame({'Survived': predictions})
#Prediction.head()
#lr_preds = LR.predict(X_test).astype(int)

In [ ]:
output = pd.DataFrame({'PassengerId': train.PassengerId, 'Survived': Prediction["Survived"]})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

# DONE!!